In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [20]:
df = pd.read_excel(r"C:\Users\Monster\Desktop\konut-fiyat-tahmin-projesi\Veri Seti\istanbul_konut2.xlsx")

In [21]:
# Eksik değerlerin toplamını ve oranını göster
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100

# Eksik değer tablosu oluştur
missing_data = pd.DataFrame({'Eksik Değerler': missing_values, 'Eksik Oranı (%)': missing_percent})
print(missing_data)

               Eksik Değerler  Eksik Oranı (%)
Fiyat                       0         0.000000
İlce                        0         0.000000
Mahalle                     0         0.000000
Metre Kare                  0         0.000000
Oda Sayısı                  2         0.006988
Yaş                        24         0.083852
Bulunduğu Kat             936         3.270212


In [22]:
# 'Yaş' sütunundaki eksik değerleri medyan ile doldur
df['Yaş'] = df['Yaş'].fillna(df['Yaş'].median())
# 'Bulunduğu Kat' sütunundaki eksik değerleri mod ile doldur
df['Bulunduğu Kat'] = df['Bulunduğu Kat'].fillna(df['Bulunduğu Kat'].mode()[0])
# Oda Sayısı sütunundaki eksik değerleri mod (en sık görülen değer) ile doldur
most_frequent_value = df['Oda Sayısı'].mode()[0]
df['Oda Sayısı'] = df['Oda Sayısı'].fillna(most_frequent_value)
print(df.isnull().sum())

Fiyat            0
İlce             0
Mahalle          0
Metre Kare       0
Oda Sayısı       0
Yaş              0
Bulunduğu Kat    0
dtype: int64


In [23]:
df

,Fiyat,İlce,Mahalle,Metre Kare,Oda Sayısı,Yaş,Bulunduğu Kat
0,5000000,Adalar,Burgazada Mah,55,2.0,24.0,0.0
1,7500000,Adalar,Nizam Mah,65,2.0,31.0,0.0
2,4750000,Adalar,Nizam Mah,60,2.0,34.0,0.0
3,7000000,Adalar,MadeMah,75,2.0,35.0,0.0
4,4200000,Adalar,Nizam Mah,51,2.0,38.0,0.0
...,...,...,...,...,...,...,...
28617,4150000,Zeytinburnu,VeliefendMah.,150,6.0,29.0,1.0
28618,4500000,Zeytinburnu,VeliefendMah.,170,6.0,30.0,1.0
28619,17000000,Zeytinburnu,Gökalp Mah.,350,12.0,16.0,0.0
28620,19000000,Zeytinburnu,Beştelsiz Mah.,500,12.0,28.0,0.0


# 1-Encoding (İlçe ve Mahalle Verilerini Sayısal Değerlere Döndürme)


# A- Lable Encoding:
##Her bir benzersiz ilçe ve mahalle adına bir sayısal değer atanır (örneğin, Adalar: 0, Avcılar: 1)

In [24]:
df

,Fiyat,İlce,Mahalle,Metre Kare,Oda Sayısı,Yaş,Bulunduğu Kat
0,5000000,Adalar,Burgazada Mah,55,2.0,24.0,0.0
1,7500000,Adalar,Nizam Mah,65,2.0,31.0,0.0
2,4750000,Adalar,Nizam Mah,60,2.0,34.0,0.0
3,7000000,Adalar,MadeMah,75,2.0,35.0,0.0
4,4200000,Adalar,Nizam Mah,51,2.0,38.0,0.0
...,...,...,...,...,...,...,...
28617,4150000,Zeytinburnu,VeliefendMah.,150,6.0,29.0,1.0
28618,4500000,Zeytinburnu,VeliefendMah.,170,6.0,30.0,1.0
28619,17000000,Zeytinburnu,Gökalp Mah.,350,12.0,16.0,0.0
28620,19000000,Zeytinburnu,Beştelsiz Mah.,500,12.0,28.0,0.0


In [25]:
from sklearn.preprocessing import LabelEncoder

# İlçe ve Mahalle sütunlarını encode edin
label_encoder = LabelEncoder()
df['Ilce_encoded'] = label_encoder.fit_transform(df['İlce'])
df['Mahalle_encoded'] = label_encoder.fit_transform(df['Mahalle'])

In [26]:
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Özellikler (X) ve hedef değişkeni (y) belirleyelim
X = df[['Metre Kare', 'Oda Sayısı', 'Yaş', 'Bulunduğu Kat', 'Ilce_encoded', 'Mahalle_encoded']]
y = df['Fiyat']

# Veriyi eğitim ve test olarak ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest modelini oluştur
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Modeli eğit
model.fit(X_train, y_train)

# Test verisi üzerinde tahmin yap
y_pred = model.predict(X_test)

# R² skorunu hesapla
r2 = r2_score(y_test, y_pred)

# Sonucu yazdır
print("Veri önişleme yapılmamış verisetinin R² değeri sonucu (Random Forest):")
print("R² Score:", r2)

Veri önişleme yapılmamış verisetinin R² değeri sonucu (Random Forest):
R² Score: 0.8203713983140706


In [28]:
df

,Fiyat,İlce,Mahalle,Metre Kare,Oda Sayısı,Yaş,Bulunduğu Kat,Ilce_encoded,Mahalle_encoded
0,5000000,Adalar,Burgazada Mah,55,2.0,24.0,0.0,0,97
1,7500000,Adalar,Nizam Mah,65,2.0,31.0,0.0,0,319
2,4750000,Adalar,Nizam Mah,60,2.0,34.0,0.0,0,319
3,7000000,Adalar,MadeMah,75,2.0,35.0,0.0,0,282
4,4200000,Adalar,Nizam Mah,51,2.0,38.0,0.0,0,319
...,...,...,...,...,...,...,...,...,...
28617,4150000,Zeytinburnu,VeliefendMah.,150,6.0,29.0,1.0,32,427
28618,4500000,Zeytinburnu,VeliefendMah.,170,6.0,30.0,1.0,32,427
28619,17000000,Zeytinburnu,Gökalp Mah.,350,12.0,16.0,0.0,32,175
28620,19000000,Zeytinburnu,Beştelsiz Mah.,500,12.0,28.0,0.0,32,87


# B- One-Hot Encoding:

In [34]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


# OneHotEncoder nesnesini oluştur
# handle_unknown='ignore' parametresi, eğitimde görülmeyen kategorileri yok sayar
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)


# ColumnTransformer ile hangi sütunların dönüştürüleceğini belirtiyoruz
# Diğer sütunlar olduğu gibi kalacak (remainder='passthrough')
ct = ColumnTransformer(
    [('onehot', ohe, ['İlce', 'Mahalle'])],
    remainder='passthrough'
)

# DataFrame'i dönüştür
df_transformed = ct.fit_transform(df)

# Dönüştürülmüş veriyi DataFrame'e çevirme (isteğe bağlı)
# Sütun isimlerini almak için:
feature_names = ct.get_feature_names_out()
df_transformed = pd.DataFrame(df_transformed, columns=feature_names)

In [35]:
df_transformed

,onehot__İlce_Adalar,onehot__İlce_Arnavutköy,onehot__İlce_Ataşehir,onehot__İlce_Avcılar,onehot__İlce_Bahçelievler,onehot__İlce_Bakırköy,onehot__İlce_Bayramp,onehot__İlce_Bağcılar,onehot__İlce_Başakşe,onehot__İlce_Beykoz,...,onehot__Mahalle_Şirintepe Mah.,onehot__Mahalle_şilvadMah.,onehot__Mahalle_şrutiyet Mah.,remainder__Fiyat,remainder__Metre Kare,remainder__Oda Sayısı,remainder__Yaş,remainder__Bulunduğu Kat,remainder__Ilce_encoded,remainder__Mahalle_encoded
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5000000.0,55.0,2.0,24.0,0.0,0.0,97.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,7500000.0,65.0,2.0,31.0,0.0,0.0,319.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4750000.0,60.0,2.0,34.0,0.0,0.0,319.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,7000000.0,75.0,2.0,35.0,0.0,0.0,282.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4200000.0,51.0,2.0,38.0,0.0,0.0,319.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28617,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4150000.0,150.0,6.0,29.0,1.0,32.0,427.0
28618,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4500000.0,170.0,6.0,30.0,1.0,32.0,427.0
28619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,17000000.0,350.0,12.0,16.0,0.0,32.0,175.0
28620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,19000000.0,500.0,12.0,28.0,0.0,32.0,87.0


In [33]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# OneHotEncoder nesnesini oluştur
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

# ColumnTransformer ile hangi sütunların dönüştürüleceğini belirtiyoruz
ct = ColumnTransformer(
    [('onehot', ohe, ['İlce', 'Mahalle'])],
    remainder='passthrough'  # Diğer sütunlar olduğu gibi kalsın
)

# Veriyi eğitim ve test kümelerine ayır
X = df.drop('Fiyat', axis=1)
y = df['Fiyat']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# One-Hot Encoding'i eğitim ve test kümelerine uygula
X_train_transformed = ct.fit_transform(X_train)
X_test_transformed = ct.transform(X_test)

# Random Forest modelini oluştur ve eğit
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_transformed, y_train)

# Test verisi üzerinde tahmin yap
y_pred = model.predict(X_test_transformed)

# R² skorunu hesapla
r2 = r2_score(y_test, y_pred)

# Sonucu yazdır
print("One-Hot Encoding uygulanmış veri seti ile R² değeri (Random Forest):")
print("R² Score:", r2)

One-Hot Encoding uygulanmış veri seti ile R² değeri (Random Forest):
R² Score: 0.822527474579572


# C- Frekans Encoding (Frequency Encoding)

##Frequency Encoding, kategorik değişkenlerdeki her bir değeri, o değerin veri setindeki frekansıyla (tekrar sayısı) değiştiren bir tekniktir

In [36]:
# Frequency Encoding uygulama
def frequency_encode(df, column):
    frequencies = df[column].value_counts(normalize=True)
    return df[column].map(frequencies)

df['Ilce_encoded'] = frequency_encode(df, 'İlce')
df['Mahalle_encoded'] = frequency_encode(df, 'Mahalle')

# Özellikler (X) ve hedef değişkeni (y) belirleyelim
X = df[['Metre Kare', 'Oda Sayısı', 'Yaş', 'Bulunduğu Kat', 'Ilce_encoded', 'Mahalle_encoded']]
y = df['Fiyat']

# Veriyi eğitim ve test olarak ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest modelini oluştur ve eğit
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Test verisi üzerinde tahmin yap
y_pred = model.predict(X_test)

# R² skorunu hesapla
r2 = r2_score(y_test, y_pred)

# Sonucu yazdır
print("Frequency Encoding uygulanmış veri seti ile R² değeri (Random Forest):")
print("R² Score:", r2)

Frequency Encoding uygulanmış veri seti ile R² değeri (Random Forest):
R² Score: 0.8173914252474903


# D- Target Encoding (Mean Encoding)

##Her bir ilçe/mahalle, o ilçe/mahalledeki ortalama konut fiyatı ile değiştirilir

In [37]:
# Target Encoding uygulama
def target_encode(df, target_col):
    # İlce için ortalama fiyatları hesapla
    ilce_means = df.groupby('İlce')[target_col].mean()
    df['Ilce_encoded'] = df['İlce'].map(ilce_means)

    # Mahalle için ortalama fiyatları hesapla
    mahalle_means = df.groupby('Mahalle')[target_col].mean()
    df['Mahalle_encoded'] = df['Mahalle'].map(mahalle_means)

    return df

# Target Encoding'i uygula
df_encoded = target_encode(df.copy(), 'Fiyat')

# Özellikler (X) ve hedef değişkeni (y) belirleyelim
X = df_encoded[['Metre Kare', 'Oda Sayısı', 'Yaş', 'Bulunduğu Kat', 'Ilce_encoded', 'Mahalle_encoded']]
y = df_encoded['Fiyat']

# Veriyi eğitim ve test olarak ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest modelini oluştur ve eğit
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Test verisi üzerinde tahmin yap
y_pred = model.predict(X_test)

# R² skorunu hesapla
r2 = r2_score(y_test, y_pred)

# Sonucu yazdır
print("Target Encoding uygulanmış veri seti ile R² değeri (Random Forest):")
print("R² Score:", r2)

Target Encoding uygulanmış veri seti ile R² değeri (Random Forest):
R² Score: 0.8296936501480221


In [38]:
df_encoded

,Fiyat,İlce,Mahalle,Metre Kare,Oda Sayısı,Yaş,Bulunduğu Kat,Ilce_encoded,Mahalle_encoded
0,5000000,Adalar,Burgazada Mah,55,2.0,24.0,0.0,1.047431e+07,1.064654e+07
1,7500000,Adalar,Nizam Mah,65,2.0,31.0,0.0,1.047431e+07,1.104514e+07
2,4750000,Adalar,Nizam Mah,60,2.0,34.0,0.0,1.047431e+07,1.104514e+07
3,7000000,Adalar,MadeMah,75,2.0,35.0,0.0,1.047431e+07,9.944185e+06
4,4200000,Adalar,Nizam Mah,51,2.0,38.0,0.0,1.047431e+07,1.104514e+07
...,...,...,...,...,...,...,...,...,...
28617,4150000,Zeytinburnu,VeliefendMah.,150,6.0,29.0,1.0,4.050061e+06,1.965306e+06
28618,4500000,Zeytinburnu,VeliefendMah.,170,6.0,30.0,1.0,4.050061e+06,1.965306e+06
28619,17000000,Zeytinburnu,Gökalp Mah.,350,12.0,16.0,0.0,4.050061e+06,4.250833e+06
28620,19000000,Zeytinburnu,Beştelsiz Mah.,500,12.0,28.0,0.0,4.050061e+06,5.503889e+06


# E- Hibrid Yöntem

##Label Encoding + Frequency Encoding kombinasyonu

In [40]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Label Encoding ve Frequency Encoding fonksiyonlarını tanımlayalım
def label_and_frequency_encode(df, column):
    """
    Verilen kolona önce Label Encoding, sonra Frequency Encoding uygular.
    
    Parameters:
    df: DataFrame
    column: Label ve Frequency encoding uygulanacak kolon adı
    
    Returns:
    encoded_labels: Label encoded değerler
    encoded_frequencies: Frequency encoded değerler
    label_encoder: Fit edilmiş LabelEncoder objesi
    """
    # Label Encoding
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(df[column])
    
    # Frequency Encoding
    value_counts = pd.Series(encoded_labels).value_counts(normalize=True)
    encoded_frequencies = pd.Series(encoded_labels).map(value_counts)
    
    return encoded_labels, encoded_frequencies, label_encoder

# Ana işlem pipeline'ı
def process_data(df):
    """
    DataFrame'e encoding işlemlerini uygular ve model için hazırlar
    """
    # İlçe encoding
    ilce_labels, ilce_freq, ilce_encoder = label_and_frequency_encode(df, 'İlce')
    df['Ilce_encoded'] = ilce_labels
    df['Ilce_freq'] = ilce_freq
    
    # Mahalle encoding
    mahalle_labels, mahalle_freq, mahalle_encoder = label_and_frequency_encode(df, 'Mahalle')
    df['Mahalle_encoded'] = mahalle_labels
    df['Mahalle_freq'] = mahalle_freq
    
    # Model için feature'ları seç
    X = df[[
        'Metre Kare',
        'Oda Sayısı',
        'Yaş',
        'Bulunduğu Kat',
        'Ilce_encoded',
        'Mahalle_encoded',
        'Ilce_freq',
        'Mahalle_freq'
    ]]
    y = df['Fiyat']
    
    return X, y, ilce_encoder, mahalle_encoder

# Veriyi hazırla ve modeli eğit
def train_model(df):
    """
    Veriyi hazırlar ve Random Forest modelini eğitir
    """
    # Veriyi hazırla
    X, y, ilce_encoder, mahalle_encoder = process_data(df)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Model oluştur ve eğit
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Test seti üzerinde tahmin yap
    y_pred = model.predict(X_test)
    
    # Performans metriklerini hesapla
    r2 = r2_score(y_test, y_pred)
    
    return {
        'model': model,
        'r2_score': r2,
        'ilce_encoder': ilce_encoder,
        'mahalle_encoder': mahalle_encoder,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test
    }

# Kullanım örneği:

results = train_model(df)
print(f"R² Score: {results['r2_score']:.4f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': results['X_train'].columns,
    'importance': results['model'].feature_importances_
}).sort_values('importance', ascending=False)
print("\nFeature Importance:")
print(feature_importance)

R² Score: 0.8288

Feature Importance:
           feature  importance
0       Metre Kare    0.327645
6        Ilce_freq    0.184491
3    Bulunduğu Kat    0.164281
4     Ilce_encoded    0.112233
2              Yaş    0.082223
5  Mahalle_encoded    0.053960
7     Mahalle_freq    0.050849
1       Oda Sayısı    0.024317
